# Imports

In [33]:
import dill
import joblib
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report
from sqlalchemy import create_engine

import credentials

# Load the test data

The model was trained with the target inverted to make recall more meaningful to the problem and I couldn't for the life of me figure out how to include that in the pipeline. So I'll just invert the target here.

In [34]:
db_con = create_engine(credentials.DB_URL)

test_df = pd.read_sql('SELECT * FROM raw.test_set', db_con)
test_df['Potability'] = 1 - test_df.Potability
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 656 entries, 0 to 655
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ph               568 non-null    float64
 1   Hardness         656 non-null    float64
 2   Solids           656 non-null    float64
 3   Chloramines      656 non-null    float64
 4   Sulfate          488 non-null    float64
 5   Conductivity     656 non-null    float64
 6   Organic_carbon   656 non-null    float64
 7   Trihalomethanes  625 non-null    float64
 8   Turbidity        656 non-null    float64
 9   Potability       656 non-null    int64  
dtypes: float64(9), int64(1)
memory usage: 51.4 KB


# Load the model

The pipeline contains a custom transformer function that I saved along with the pipeline itself. Those both get loaded from the same file all that's needed is to make sure the right imports are included in the notebook.

In [35]:
model, preprocess_transform = joblib.load('final_pipe.joblib')
predictions = model.predict(test_df)
print(classification_report(test_df['Potability'], predictions))

              precision    recall  f1-score   support

           0       0.82      0.27      0.40       245
           1       0.69      0.97      0.80       411

    accuracy                           0.70       656
   macro avg       0.76      0.62      0.60       656
weighted avg       0.74      0.70      0.65       656



These performance metrics are even better than what was seen with the cross-validation predictions on the training data. It's probably due to using the whole training set to train the model.

In [36]:
pred_proba = model.predict_proba(test_df)
print(pred_proba)

[[0.48779982 0.51220018]
 [0.6292127  0.3707873 ]
 [0.5844335  0.4155665 ]
 ...
 [0.5453868  0.4546132 ]
 [0.40652743 0.59347257]
 [0.22500859 0.77499141]]


In [38]:
pred_proba_df = pd.DataFrame(pred_proba, columns=['Probability_0', 'Probability_1'])
pred_proba_df.to_sql('test_set_predictions', db_con, if_exists='replace', schema='analytics', index=False, method='multi', chunksize=10)

656

# Summary

In the end, the model's performance is a little weaker than what is desired by a water testing device. In order for the device to be trusted, I'd want an accuracy in the high 90s. The recall is okay, but any missed not potible samples is too many. 

I did learn a lot doing this assignment though. This was my first time making a full raw data to prediction pipeline and figuring out how to make the custom transformer and use it in a saved model was a fun challenge. I kept more than necessary features last week to give the model more flexibility when searching through hyperparameters, but then once the hyperparmeters and model were chosen, I removed quite a few features to make the model smaller and I think better. I think if you have the computation resources having a few extra features to train with is good, but then once you have the model you want, removing features can make the model more compact and faster to run.

I've had luck in the past with a voting ensemble of models and I thought taking the top 3 models to make one would produce the best model, but that wasn't the case. Maybe the 2 weaker models were just draggin the SVC down. 

I opted to use cross-validation as opposed to a dedicated validation set because I wanted to use all of the data to get performance metrics. Besides, I think cross-validation is a better method when computation resources are permitting. Using 5-fold cross validation is like having 5 validation sets and taking the mean results from those 5 sets gives a much less biased estimate of the model's performance.